# COGS 108 - Data Checkpoint

# Names

- Sarai Mendoza
- Aretha Li
- Scott Shyu
- Jing Yuan
- Ramtin Kazemi


<a id='research_question'></a>
# Research Question

Among unemployment rate, crime rate, and education funding, which variable has the strongest relationship with suicide rates in major U.S. state?


# Dataset(s)

Dataset #1: 

- Dataset Name: State Health Compare (Suicide per 100,000 from 1999-2020 by state) 
    - Link to the dataset: http://statehealthcompare.shadac.org/table/211/suicide-deaths-per-100000-people-by-total#1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52/a/32/243 
    - Number of observations: 1,122
    - This dataset shows the suicide rate for all 50 states plus the District of Columbia and spans from 1999-2020. They express the suicide rate by suicide per 100,000 and is calculated using data from the CDC. 

Dataset #2: 
- Dataset Name: Crime Rate and Unemployment Rate by state (1976-2014)
    - Link to the dataset: https://www.kaggle.com/lydiavasil/crime-rate-and-unemployment-rate-by-state 
    - Number of observations: 1989
    - This dataset shows the crime rate and unemployment rate by state including the District of Columbia, starting from 1976 to 2014. The crimes in the dataset are murder, rape, robbery, and aggravated assault. 
    
Dataset #3: 
- Dataset Name: U.S. Education Dataset: Unionification Process (1986-2019) 
    - Link to the dataset: https://www.kaggle.com/noriuk/us-education-datasets-unification-project  
    - Number of observations: 1,715 
    - This dataset shows multiple facets of U.S. education data such as the amount of students enrolled each year, school’s federal, state, and local revenue, as well as instruction expenditures, student scores, etc. These data will be used to support if there is a relationship between the enrollment of students and school revenue in K-12 education to the crime and unemployment rate across different states.
  
  
Combining Datasets: The datasets about crime rates, unemployment rates, and educational funding in U.S. states will be combined and analyzed in order to figure out which of these variables would have the strongest relationship with suicide rates in U.S. states.  




 


# Setup

In [1]:
import statistics
import numpy as np
import pandas as pd
import requests
import json
import seaborn as sns
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.style as style
import patsy
import statsmodels.api as sm
import scipy.stats as stats
from scipy.stats import ttest_ind, chisquare, normaltest
from collections import defaultdict

plt.rcParams['figure.figsize'] = (12, 6)
sns.set(font_scale=1.5, style="white")

In [2]:
suicide_rate = pd.read_csv('suicide_rate.csv')
crime_unemployment_rate = pd.read_csv('crimebystatecombinedwithunemployment.csv')
education = pd.read_csv('states_all.csv')

In [3]:
suicide_rate.head()

,Fips,Location,TimeFrame,Data Type,Data,MOE
0,1,Alabama,1999,"Rate per 100,000",12.46094,1.037785
1,1,Alabama,2000,"Rate per 100,000",13.10666,1.064901
2,1,Alabama,2001,"Rate per 100,000",11.37451,0.986649
3,1,Alabama,2002,"Rate per 100,000",11.44968,0.992287
4,1,Alabama,2003,"Rate per 100,000",11.43737,0.985613


In [4]:
crime_unemployment_rate.head()

,state,unemployment,year,Population,violent total,Murder,rape,Robbery,Aggravated assault,property total,Burglary,Larceny theft,vehicle theft
0,AL,6.808,1976,3665000,388.8,15.1,21.7,96.0,256.0,3419.5,1170.0,1987.2,262.3
1,AL,7.325,1977,3690000,414.4,14.2,25.2,96.8,278.3,3298.2,1135.5,1881.9,280.7
2,AL,6.380,1978,3742000,419.1,13.3,25.5,99.1,281.2,3519.7,1229.3,1987.9,302.5
3,AL,7.158,1979,3769000,413.3,13.2,27.5,109.5,263.1,3830.5,1287.3,2223.2,320.1
4,AL,8.867,1980,3861466,448.5,13.2,30.0,132.1,273.2,4485.1,1526.7,2642.2,316.2


In [5]:
education.head()

,PRIMARY_KEY,STATE,YEAR,ENROLL,TOTAL_REVENUE,FEDERAL_REVENUE,STATE_REVENUE,LOCAL_REVENUE,TOTAL_EXPENDITURE,INSTRUCTION_EXPENDITURE,...,GRADES_4_G,GRADES_8_G,GRADES_12_G,GRADES_1_8_G,GRADES_9_12_G,GRADES_ALL_G,AVG_MATH_4_SCORE,AVG_MATH_8_SCORE,AVG_READING_4_SCORE,AVG_READING_8_SCORE
0,1992_ALABAMA,ALABAMA,1992,NaN,2678885.0,304177.0,1659028.0,715680.0,2653798.0,1481703.0,...,57948.0,58025.0,41167.0,NaN,NaN,731634.0,208.0,252.0,207.0,NaN
1,1992_ALASKA,ALASKA,1992,NaN,1049591.0,106780.0,720711.0,222100.0,972488.0,498362.0,...,9748.0,8789.0,6714.0,NaN,NaN,122487.0,NaN,NaN,NaN,NaN
2,1992_ARIZONA,ARIZONA,1992,NaN,3258079.0,297888.0,1369815.0,1590376.0,3401580.0,1435908.0,...,55433.0,49081.0,37410.0,NaN,NaN,673477.0,215.0,265.0,209.0,NaN
3,1992_ARKANSAS,ARKANSAS,1992,NaN,1711959.0,178571.0,958785.0,574603.0,1743022.0,964323.0,...,34632.0,36011.0,27651.0,NaN,NaN,441490.0,210.0,256.0,211.0,NaN
4,1992_CALIFORNIA,CALIFORNIA,1992,NaN,26260025.0,2072470.0,16546514.0,7641041.0,27138832.0,14358922.0,...,418418.0,363296.0,270675.0,NaN,NaN,5254844.0,208.0,261.0,202.0,NaN


# Data Cleaning

Describe your data cleaning steps here.

In [6]:
suicide_rate= suicide_rate.dropna()
#clean and years are ok
suicide_rate=suicide_rate.rename(columns={'TimeFrame': 'year','Data': 'rate'})
suicide_rate=suicide_rate.loc[suicide_rate['year'] < 2015]
suicide_rate=suicide_rate.loc[suicide_rate['year'] > 1998]
suicide_rate=suicide_rate.reset_index()
suicide_rate=suicide_rate[['Location','year','rate']]
suicide_rate.head()

,Location,year,rate
0,Alabama,1999,12.46094
1,Alabama,2000,13.10666
2,Alabama,2001,11.37451
3,Alabama,2002,11.44968
4,Alabama,2003,11.43737


In [7]:
crime_unemployment_rate=crime_unemployment_rate.dropna()
#clean and years are ok
crime_unemployment_rate=crime_unemployment_rate.loc[crime_unemployment_rate['year'] < 2015]
crime_unemployment_rate=crime_unemployment_rate.loc[crime_unemployment_rate['year'] > 1998]
crime_unemployment_rate=crime_unemployment_rate.reset_index()
crime_unemployment_rate=crime_unemployment_rate.drop(['index'], axis = 1)
crime_unemployment_rate.head()

,state,unemployment,year,Population,violent total,Murder,rape,Robbery,Aggravated assault,property total,Burglary,Larceny theft,vehicle theft
0,AL,4.733,1999,4369862,490.2,7.9,34.6,121.2,326.5,3922.3,884.4,2737.3,300.6
1,AL,4.583,2000,4447100,486.2,7.4,33.3,128.2,317.2,4059.7,906.9,2864.8,288.0
2,AL,5.125,2001,4468912,438.2,8.5,30.6,125.0,274.1,3876.8,909.4,2685.0,282.4
3,AL,5.900,2002,4478896,445.0,6.8,37.2,133.1,268.0,4027.8,950.6,2767.0,310.1
4,AL,6.017,2003,4503726,429.2,6.6,36.8,134.1,251.7,4046.4,960.2,2754.1,332.1


In [8]:
educationExp = education[['STATE','YEAR','TOTAL_EXPENDITURE','INSTRUCTION_EXPENDITURE']]

In [9]:
#educationExp=educationExp.dropna(axis=0,how='all')
#educationExp.isna().value_counts()
#1999-2014
#nans = educationExp[educationExp.isna().any(axis=1)]

educationExp=educationExp.loc[educationExp['YEAR'] < 2015]
educationExp=educationExp.loc[educationExp['YEAR'] > 1998]
educationExp.isna().value_counts()
educationExp=educationExp.dropna()
educationExp=educationExp.reset_index()
educationExp=educationExp.drop(['index'], axis = 1)
educationExp.head()

,STATE,YEAR,TOTAL_EXPENDITURE,INSTRUCTION_EXPENDITURE
0,ALABAMA,1999,4622810.0,2399626.0
1,ALASKA,1999,1289988.0,705069.0
2,ARIZONA,1999,5072471.0,2272859.0
3,ARKANSAS,1999,2650808.0,1424443.0
4,CALIFORNIA,1999,40834643.0,21110543.0


In [10]:
unemp_rate = crime_unemployment_rate[['state','year','unemployment']]
unemp_rate.head()

,state,year,unemployment
0,AL,1999,4.733
1,AL,2000,4.583
2,AL,2001,5.125
3,AL,2002,5.900
4,AL,2003,6.017


In [11]:
violent_rate = crime_unemployment_rate[['state','year','violent total']]
violent_rate.head()

,state,year,violent total
0,AL,1999,490.2
1,AL,2000,486.2
2,AL,2001,438.2
3,AL,2002,445.0
4,AL,2003,429.2


The data for the unemployment and crime rate and suicide rate were clean but the data set on education was not. In order to have clean data, we had to see how many rows had missing values and then dropped them. In addition to dropping missing values, we had to slice and create a variable that only stored the data for the years we were interested in 1999-2014. Because 2 of our variables were in one single data frame, we had to create a new data frame that only contained the unemployment rates and one with the crime rate.